In [117]:
SEASON = '2015-16'

In [118]:
import pandas as pd
import myconstants as c
from scipy.stats import zscore

NUM_GAMES = c.NUM_GAMES[SEASON]
MIN_GAMES = NUM_GAMES * 0.60

p_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_PerGame.csv')
t_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Team_Standings.csv')

# join player per game and advanced stats
p_advanced_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_Advanced.csv')
p_advanced_df = p_advanced_df.drop(columns=['Player', 'Tms', 'Pos', 'Age', 'G', 'MP'])
p_df = p_df.merge(p_advanced_df, on='PlayerID')

# populate team df with team codes
t_df['TeamCode'] = t_df['Team'].apply(lambda x: c.TEAM_CODE[x])

# average team wins per player
def average_wins(tms):
    tms_list = tms.split(',')
    tms_wins = []
    for t in tms_list:
        tms_wins.append(t_df.loc[t_df['TeamCode'] == t]['OverallW'].values[0])
    return sum(tms_wins) / len(tms_wins)
p_df['TmsWins'] = p_df['Tms'].apply(average_wins)

# players who haven't played enough games don't qualify
p_df = p_df.loc[p_df['G'] >= MIN_GAMES]

# ranking algorithm
p_df['PlayerScore'] = 1.5*zscore(p_df['PTS'] + p_df['TRB'] + p_df['AST'] + p_df['STL'] + p_df['BLK']) + zscore(p_df['WS']) + zscore(p_df['VORP']) + zscore(p_df['BPM']) + zscore(p_df['TmsWins'])
p_df['PlayerScore'] = p_df['PlayerScore'] + c.PLAYER_SCORE_ADDITION # more positive PlayerScores
p_df['PlayerScore'] = p_df['PlayerScore'].apply(lambda x: pow(x, c.PLAYER_SCORE_EXPONENT) if x > 1 else x) # make higher PlayerScores more significant
p_df = p_df.sort_values('PlayerScore', ascending=False)

# Move [PlayerScore] to the right of [Pos], drop PlayerID, index rows starting from 1, stylistic choice
col = p_df.pop('PlayerScore')
p_df.insert(p_df.columns.get_loc('Pos') + 1, col.name, col)
p_df = p_df.drop('PlayerID', axis=1)
p_df = p_df.reset_index(drop=True)
p_df.index = p_df.index + 1

p_df.head(50)

,Player,Tms,Pos,PlayerScore,Age,G,GS,MP,FG,FGA,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,TmsWins
1,Stephen Curry,GSW,PG,149.216976,27,79,79,34.2,10.2,20.2,...,32.6,13.8,4.1,17.9,0.318,10.3,1.6,11.9,9.5,73.0
2,Kevin Durant,OKC,SF,114.600175,27,72,72,35.8,9.7,19.2,...,30.6,11.0,3.5,14.5,0.270,8.4,1.6,9.9,7.8,55.0
3,LeBron James,CLE,SF,106.825534,31,76,76,35.6,9.7,18.6,...,31.4,9.6,4.0,13.6,0.242,7.0,2.0,9.0,7.5,57.0
4,Russell Westbrook,OKC,PG,103.390663,27,80,80,34.4,8.2,18.1,...,31.6,10.0,4.0,14.0,0.245,6.4,1.4,7.8,6.8,55.0
5,Kawhi Leonard,SAS,SF,98.622103,24,72,72,33.1,7.7,15.1,...,25.8,8.3,5.5,13.7,0.277,6.1,3.0,9.1,6.7,67.0
6,James Harden,HOU,SG,92.517695,26,82,82,38.1,8.7,19.7,...,32.5,10.7,2.6,13.3,0.204,6.4,0.2,6.6,6.8,41.0
7,Chris Paul,LAC,PG,86.206709,30,74,74,32.7,7.0,15.1,...,27.1,9.2,3.5,12.7,0.253,5.9,2.0,7.9,6.0,53.0
8,Draymond Green,GSW,PF,80.654926,25,81,81,34.7,5.0,10.1,...,18.8,6.0,5.1,11.1,0.190,2.6,2.9,5.5,5.3,73.0
9,Kyle Lowry,TOR,PG,77.264827,29,77,77,37.0,6.6,15.6,...,26.1,8.0,3.7,11.6,0.196,4.7,1.1,5.8,5.7,56.0
10,Paul George,IND,SF,63.647900,25,81,81,34.8,7.5,17.9,...,30.4,4.4,4.8,9.2,0.157,3.7,1.2,4.9,4.9,45.0
